In [31]:
year = '2019'

In [32]:
import csv
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re
import pandas as pd
from sqlalchemy import create_engine

In [41]:
schools_list = []
school_urls = []
teams = []

schools_2019_file = f'db/{year}_schools.csv'
with open(schools_2019_file, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        schools_list.append(row[0])
        school_urls.append(row[1])

In [42]:
print(school_urls)

['duke', 'north-carolina-central', 'north-dakota-state', 'virginia-commonwealth', 'central-florida', 'mississippi-state', 'liberty', 'virginia-tech', 'saint-louis', 'maryland', 'belmont', 'temple', 'louisiana-state', 'yale', 'louisville', 'minnesota', 'michigan-state', 'bradley', 'gonzaga', 'fairleigh-dickinson', 'prairie-view', 'syracuse', 'baylor', 'marquette', 'murray-state', 'florida-state', 'vermont', 'buffalo', 'arizona-state', 'st-johns-ny', 'texas-tech', 'northern-kentucky', 'nevada', 'florida', 'michigan', 'montana', 'virginia', 'gardner-webb', 'mississippi', 'oklahoma', 'wisconsin', 'oregon', 'kansas-state', 'california-irvine', 'villanova', 'saint-marys-ca', 'purdue', 'old-dominion', 'cincinnati', 'iowa', 'tennessee', 'colgate', 'north-carolina', 'iona', 'utah-state', 'washington', 'auburn', 'new-mexico-state', 'kansas', 'northeastern', 'iowa-state', 'ohio-state', 'houston', 'georgia-state', 'wofford', 'seton-hall', 'kentucky', 'abilene-christian']


In [43]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [44]:
def ordinal_to_number(ordinal):
    number = re.sub('[stndrdh]','',ordinal)
    return number

In [45]:
base_schools_url = 'https://www.sports-reference.com/cbb/schools/'

for index, url in enumerate(school_urls):
    team = {'name': schools_list[index]}
    
    team_url = f'{base_schools_url}{url}/{year}.html'
    browser.visit(team_url)
    team_soup = bs(browser.html,'html.parser')
    
    team_meta_table = team_soup.find('div', attrs={'data-template': 'Partials/Teams/Summary'})
    team_meta_info = team_meta_table.find_all('p')
    
    for info in team_meta_info:
        if 'SRS' in info.text:
            team['team_SRS'] = re.sub('[SRS: ]','',info.text).split('(')[0]
            team['team_rank_SRS'] = ordinal_to_number(re.sub('[SRS: ]','',info.text).split('of')[0].split('(')[1])
        elif 'SOS' in info.text:
            team['team_SOS'] = re.sub('[SOS: ]','',info.text).split('(')[0]
            team['team_rank_SOS'] = ordinal_to_number(re.sub('[SOS: ]','',info.text).split('of')[0].split('(')[1])

    stats_table = team_soup.find('table', id='team_stats')
    stat_table_rows = stats_table.find('tbody').find_all('tr')

    team_stats = stat_table_rows[0].find_all('td')
    team_rank = stat_table_rows[1].find_all('td')
    opp_stats = stat_table_rows[2].find_all('td')
    opp_rank = stat_table_rows[3].find_all('td')

    for stat in team_stats:
        label = 'team_'+stat['data-stat']
        value = stat.text

        team[label] = value

    for stat in team_rank:
        label = 'team_rank_'+stat['data-stat']
        value = ordinal_to_number(stat.text)

        team[label] = value

    for stat in opp_stats:
        label = 'opp_'+stat['data-stat']
        value = stat.text

        team[label] = value 

    for stat in opp_rank:
        label = 'opp_rank_'+stat['data-stat']
        value = ordinal_to_number(stat.text)

        team[label] = value
    
    teams.append(team)

In [46]:
print(teams)

[{'name': 'Duke', 'team_SRS': '28.38', 'team_rank_SRS': '1', 'team_SOS': '12.44', 'team_rank_SOS': '2', 'team_g': '34', 'team_mp': '6825', 'team_fg': '1037', 'team_fga': '2173', 'team_fg_pct': '.477', 'team_fg2': '790', 'team_fg2a': '1355', 'team_fg2_pct': '.583', 'team_fg3': '247', 'team_fg3a': '818', 'team_fg3_pct': '.302', 'team_ft': '518', 'team_fta': '751', 'team_ft_pct': '.690', 'team_orb': '455', 'team_drb': '966', 'team_trb': '1421', 'team_ast': '541', 'team_stl': '322', 'team_blk': '232', 'team_tov': '446', 'team_pf': '539', 'team_pts': '2839', 'team_pts_per_g': '83.5', 'team_rank_g': '', 'team_rank_mp': '', 'team_rank_fg': '2', 'team_rank_fga': '3', 'team_rank_fg_pct': '31', 'team_rank_fg2': '3', 'team_rank_fg2a': '16', 'team_rank_fg2_pct': '5', 'team_rank_fg3': '182', 'team_rank_fg3a': '62', 'team_rank_fg3_pct': '340', 'team_rank_ft': '47', 'team_rank_fta': '29', 'team_rank_ft_pct': '238', 'team_rank_orb': '4', 'team_rank_drb': '9', 'team_rank_trb': '2', 'team_rank_ast': '12

In [48]:
teams_df = pd.DataFrame(teams)
teams_df.head()

,name,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,...,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb
0,Duke,34,6825,418,108,806,858,637,.449,1418,...,4,99,5,8,3,265,2,322,446,1421
1,NC Central,33,6625,430,119,716,806,602,.499,1206,...,48,232,162,210,155,339,57,207,506,1227
2,North Dakota State,33,6650,395,86,824,885,641,.517,1240,...,345,26,126,155,308,26,301,159,349,1042
3,VCU,32,6450,346,81,763,678,515,.438,1175,...,115,308,209,207,30,265,124,255,446,1174
4,UCF,31,6225,388,86,725,715,498,.446,1117,...,217,63,247,184,263,51,179,178,367,1139


In [49]:
engine = create_engine("sqlite:///db/ncaa_history.sqlite")
teams_df.to_sql('teams_2019', con=engine, if_exists='replace', index=False)